In [15]:
import numpy as np
import pandas as pd
import json

In [1]:
from molar import ClientConfig, Client

admin_cfg = ClientConfig(server_url="http://localhost:8000",
                         email="default@rheed.com",
                         password="rheed",
                         database_name="main")

admin_client = Client(admin_cfg)

print(admin_client.test_token())

print(admin_client.get_alembic_revisions())

{'email': 'default@rheed.com', 'is_superuser': True, 'is_active': True, 'full_name': 'default', 'created_on': '2021-10-23T17:45:31.662744', 'user_id': 1}
       revision                                          log_entry  \
0  311e0c050706  Rev: 311e0c050706 (head)\nParent: bf3c5d811155...   
1  0bc99b5f8fcc  Rev: 0bc99b5f8fcc (head)\nParent: f31c7d486f1f...   

               branch_labels  
0               [molar-main]  
1  [compchem, eventsourcing]  


In [2]:
user_cfg = ClientConfig(server_url="http://localhost:8000",
                        email="new_user@molar.org",
                        password="new_password",
                        database_name="compchem")

user_client = Client(user_cfg)

user_client.database_creation_request(
    superuser_fullname="default",
    alembic_revisions=["compchem@head"]
)

[14:30:25] INFO     Database request created                                    ]8;id=77824;file:///Users/nfrey/miniconda3/envs/rheed-viz/lib/python3.7/site-packages/molar/client.py\client.py]8;;\:107

{'msg': 'Database request created'}

In [3]:
admin_client.approve_database("compchem")

[14:30:43] INFO     Database compchem created.                                  ]8;id=661082;file:///Users/nfrey/miniconda3/envs/rheed-viz/lib/python3.7/site-packages/molar/client.py\client.py]8;;\:107

{'msg': 'Database compchem created.'}

In [4]:
user_client.test_token()

{'email': 'new_user@molar.org',
 'is_superuser': True,
 'is_active': True,
 'full_name': 'default',
 'created_on': '2021-10-23T18:30:43.841516',
 'user_id': 1}

In [5]:
db_info = user_client.get_database_information()
db_info

,table_name,column_name,type,subtype,is_nullable,constraint_name,containt_type,references
0,calculation,calculation_id,uuid,uuid,NO,calculation_pkey,None,calculation.calculation_id
1,calculation,command_line,text,text,YES,None,None,None
2,calculation,conformer_id,uuid,uuid,NO,calculation_conformer_id_fkey,None,conformer.conformer_id
3,calculation,created_on,timestamp without time zone,timestamp,NO,None,None,None
4,calculation,input_file,text,text,YES,None,None,None
5,calculation,metadata,jsonb,jsonb,YES,None,None,None
6,calculation,output_conformer_id,uuid,uuid,YES,calculation_output_conformer_id_fkey,None,conformer.conformer_id
7,calculation,software_id,uuid,uuid,NO,calculation_software_id_fkey,None,software.software_id
8,calculation,updated_on,timestamp without time zone,timestamp,NO,None,None,None
9,conformer,atomic_numbers,ARRAY,_int4,NO,None,None,None


In [23]:
db_info[db_info.table_name == "molecule"]

,table_name,column_name,type,subtype,is_nullable,constraint_name,containt_type,references
18,molecule,created_on,timestamp without time zone,timestamp,NO,None,None,None
19,molecule,metadata,jsonb,jsonb,YES,None,None,None
20,molecule,molecule_id,uuid,uuid,NO,molecule_pkey,None,molecule.molecule_id
21,molecule,molecule_type_id,uuid,uuid,YES,molecule_molecule_type_id_fkey,None,molecule_type.molecule_type_id
22,molecule,smiles,text,text,NO,molecule_smiles_key,None,molecule.smiles
23,molecule,updated_on,timestamp without time zone,timestamp,NO,None,None,None


In [22]:
event = user_client.create_entry(
    type="molecule",
    data={
        "smiles": "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",
        "molecule_type_id": event['uuid'],
        "metadata": {"name": "cafefine"}
    }
)
event

MolarBackendError: The client got an unexpected answer from the backend:
401: Unique constraint violation!

In [25]:
event

{'type': 'molecule',
 'id': 6,
 'uuid': '4c10ba00-3d48-4cb0-9bb7-c02ede01df70',
 'event': 'create',
 'data': {'smiles': 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C',
  'metadata': {'name': 'cafefine'},
  'molecule_type_id': 'e767c83f-d04b-43b0-82ae-758814d7b8aa'},
 'timestamp': '2021-10-23T18:45:18.178994',
 'alembic_version': ['0bc99b5f8fcc'],
 'user_id': 1}

In [8]:
db_info[db_info.table_name == "numerical_data"]

,table_name,column_name,type,subtype,is_nullable,constraint_name,containt_type,references
29,numerical_data,calculation_id,uuid,uuid,NO,numerical_data_calculation_id_fkey,None,calculation.calculation_id
30,numerical_data,created_on,timestamp without time zone,timestamp,NO,None,None,None
31,numerical_data,data,ARRAY,_float8,YES,None,None,None
32,numerical_data,metadata,jsonb,jsonb,YES,None,None,None
33,numerical_data,numerical_data_id,uuid,uuid,NO,numerical_data_pkey,None,numerical_data.numerical_data_id
34,numerical_data,updated_on,timestamp without time zone,timestamp,NO,None,None,None


In [34]:
db_info[db_info.table_name == "conformer"]

,table_name,column_name,type,subtype,is_nullable,constraint_name,containt_type,references
9,conformer,atomic_numbers,ARRAY,_int4,NO,None,None,None
10,conformer,conformer_id,uuid,uuid,NO,conformer_pkey,None,conformer.conformer_id
11,conformer,created_on,timestamp without time zone,timestamp,NO,None,None,None
12,conformer,metadata,jsonb,jsonb,YES,None,None,None
13,conformer,molecule_id,uuid,uuid,YES,conformer_molecule_id_fkey,None,molecule.molecule_id
14,conformer,updated_on,timestamp without time zone,timestamp,NO,None,None,None
15,conformer,x,ARRAY,_float8,NO,None,None,None
16,conformer,y,ARRAY,_float8,NO,None,None,None
17,conformer,z,ARRAY,_float8,NO,None,None,None


In [43]:
event = user_client.create_entry(type="conformer", data={'atomic_numbers': [1, 1],
                                                        'x': [1, 1],
                                                        'y': [1, 1],
                                                        'z': [1, 1]
                                                        }
                                )
event

{'type': 'conformer',
 'id': 18,
 'uuid': 'd68b3040-59a1-4017-a8c1-af370203fbe6',
 'event': 'create',
 'data': {'x': [1, 1], 'y': [1, 1], 'z': [1, 1], 'atomic_numbers': [1, 1]},
 'timestamp': '2021-10-23T18:55:54.774976',
 'alembic_version': ['0bc99b5f8fcc'],
 'user_id': 1}

In [44]:
data = np.load('data/pca.npz')
components = data['arr_0']

In [48]:
components[0].tolist()

[2.540457228095736e-23,
 -7.871138932081763e-30,
 -1.4063380699204052e-31,
 -3.1425813517658374e-32,
 3.7115544527398075e-32,
 -1.0690792226172692e-33,
 3.421582952537175e-32,
 -6.30958293376543e-33,
 -1.543300666580634e-32,
 -3.219753635582865e-33,
 3.658876915685536e-08,
 3.658876915685536e-08,
 3.658876915685536e-08,
 3.658876915685536e-08,
 3.658876915685536e-08,
 3.658876915685536e-08,
 3.658876915685536e-08,
 3.658876915685536e-08,
 0.00016381150985436135,
 0.00016381150985436135,
 0.00016381150985436135,
 0.00016381150985436135,
 0.00016381150985436135,
 0.00016381150985436135,
 0.00016381150985436135,
 0.00016381150985436135,
 0.0001752390010801253,
 0.0001752390010801253,
 0.0001752390010801253,
 0.0001752390010801253,
 0.0001752390010801253,
 0.0001752390010801253,
 0.0001752390010801253,
 0.0001752390010801253,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [50]:
event = user_client.create_entry(type="conformer", 
                                data={"atomic_numbers": [26, 34], # iron and selenium
                                        "x": components[0].tolist(),
                                        "y": components[1].tolist(),
                                        "z": [0.],
                                        "metadata": {"samplename": "FeSe"} # "bytestring": content.decode(),}
                                        })
event

{'type': 'conformer',
 'id': 21,
 'uuid': '1aef6a8d-8710-4dae-9995-3752ac8301d9',
 'event': 'create',
 'data': {'x': [2.540457228095736e-23,
   -7.871138932081763e-30,
   -1.4063380699204052e-31,
   -3.1425813517658374e-32,
   3.7115544527398075e-32,
   -1.0690792226172692e-33,
   3.421582952537175e-32,
   -6.30958293376543e-33,
   -1.543300666580634e-32,
   -3.219753635582865e-33,
   3.658876915685536e-08,
   3.658876915685536e-08,
   3.658876915685536e-08,
   3.658876915685536e-08,
   3.658876915685536e-08,
   3.658876915685536e-08,
   3.658876915685536e-08,
   3.658876915685536e-08,
   0.00016381150985436135,
   0.00016381150985436135,
   0.00016381150985436135,
   0.00016381150985436135,
   0.00016381150985436135,
   0.00016381150985436135,
   0.00016381150985436135,
   0.00016381150985436135,
   0.0001752390010801253,
   0.0001752390010801253,
   0.0001752390010801253,
   0.0001752390010801253,
   0.0001752390010801253,
   0.0001752390010801253,
   0.0001752390010801253,
   0.0001

In [51]:
pc_df = pd.read_csv('data/pca.csv')

In [52]:
pc_df.head()

,PC1,PC2,PC3,Time,dPC1_dt
0,18.497882,9.080990,2.734563,0.000000,-117.358424
1,18.380640,9.181942,2.778275,0.000999,68.920940
2,18.635586,9.077834,2.758800,0.001998,18.044388
3,18.416693,9.067613,2.770992,0.002997,-292.743513
4,18.050684,9.144307,2.822253,0.003996,-113.738684
